# Quickstart

A tour of this package's functionality! Buckle up because it's gonna be a fun one.

## Tools and Toolkits

### Tools: `@function_tool`
You can make any function into a tool usable by the model by decorating it with `@function_tool`, but you must do the following:
- The parameters must be type annotated
- The return type *should* be str, but not required
- It must be documented with a docstring, including each parameter (most formats supported, e.g. Google-style, NumPy-style, sphinx-style, etc.)

You can use the tool from python as you normally would, but also the annotated tool will be auto-parsed and its JSON schema and argument validator (using pydantic) will be available as attributes on the function:
- `tool.schema` is the JSON schema, which you can directly pass to the OpenAI API.
- `tool.lookup` is a dict lookup table of the function name to the (auto-validated) function call, meaning you can just receive the model-generated function call and pass it directly to the tool.

Other attributes (probably commonly used):
- `tool.validator` is the pydantic validator, callable with kwargs to validate the arguments. This is what is saved in the lookup table.
- `tool.tool_enabled` is a bool switch to enable/disable the tool. This is useful esp. for Toolkits, where you can dynamically enable/disable tools based on the context.
- `tool.name` should ideally not be set directly, but passed as a kwarg to `@function_tool`. It is the name of the tool, as known to the model. If not set, it will be the name of the function.

In [1]:
from gpt_wrapper.tools import Toolkit, function_tool, fail_with_message
import logging
logger = logging.getLogger(__name__)

In [2]:
@function_tool
def print_to_console(text: str) -> str:
    '''
    Print text to console

    Args:
        text: text to print
    '''
    print(text)
    return 'success' # ideally, we always return something to tell the model

# normal call
print_to_console('Hello from python!')

# call from lookup table
lookup = print_to_console.lookup
# this would be generated by GPT
name, arguments = 'print_to_console', {'text': 'Hello from GPT!'}
func = lookup[name]
func(arguments) # notice the lack of unpacking, we pass the args dict directly

Hello from python!
Hello from GPT!


'success'

In [3]:
# if the model generates an invalid argument, it is auto-validated
print(func({'text': 123}))
print(func({'content': "Whats up?"}))

# notice that it's returned as string, because it should be passed to the model to correct itself

Invalid Argument: 1 validation error for print_to_console
text
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
Invalid Argument: 1 validation error for print_to_console
text
  Field required [type=missing, input_value={'content': 'Whats up?'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


Showing off some more goodies:
- Even async functions should seamlessly work, just don't forget to `await` them.
- `@fail_with_message(err)` is a decorator that will catch any exceptions thrown by the function and instead return the error message. This is useful for when you want to handle errors in a more graceful way than just crashing the model. It also takes an optional logger, which by default takes the `print` function, but any callable that takes a string will work, such as `logger.error` from the `logging` module.
- Usually, the `@function_tool` decorator will throw an assertion error if you forget to provide the description for any of the function or their parameters. If you really don't want to provide descriptions for some (or all), maybe because it's so self-explanatory or you need to save tokens, then you can explicitly turn off the docstring parsing by passing `@function_tool(check_description=False)`. This is not recommended, but it's there if you need it.

In [4]:
import asyncio

@function_tool(name="Fibonacci")
@fail_with_message("Error", logger=logger.error)
async def fib(n: int):
    '''
    Calculate the nth Fibonacci number

    Args:
        n: The index of the Fibonacci number to calculate
    '''
    if n < 0:
         raise ValueError("n must be >= 0")
    if n < 2:
        return n
    await asyncio.sleep(0.1)
    # return await fib(n-1) + await fib(n-2)
    # parallel
    return sum(await asyncio.gather(fib(n-1), fib(n-2)))

print(await fib(10), flush=True)
print(await fib.lookup['Fibonacci']({'n': -10}))
fib.schema

55


Tool call fib(n=-10) failed: n must be >= 0


Error: n must be >= 0


{'type': 'function',
 'function': {'name': 'Fibonacci',
  'description': 'Calculate the nth Fibonacci number',
  'parameters': {'type': 'object',
   'properties': {'n': {'description': 'The index of the Fibonacci number to calculate',
     'type': 'integer'}},
   'required': ['n']}}}

### Toolkits: `class Toolkit`
Toolkits are a collection of related function tools, esp. useful when they share a state. Also good for keeping the state bound to a single instance of the toolkit, rather than a global state.
To create a toolkit, simply subclass `Toolkit` and decorate its methods with `@function_tool`.
Enabled tools will be visible in the attributes:
- `toolkit.schema` is the JSON schema, which you can directly pass to the OpenAI API.
- `toolkit.lookup` is a dict lookup table of the function name to the (auto-validated) function call, meaning you can just receive the model-generated function call and pass it directly to the tool.

As you noticed, the interface for OpenAI API is the same for both tools and toolkits, so you can use them interchangeably, and to use multiple tools and toolkits, simply merge their schemas and lookups.

In [5]:
class Notepad(Toolkit):
    def __init__(self):
        self.content = "<Fill me in>"
    
    @function_tool
    def write(self, text: str):
        '''
        Write text to the notepad

        Args:
            text: The text to write
        '''
        self.content = text
    
    @function_tool(check_description=False)
    def read(self):
        return self.content
    
notes = Notepad()
notes.write("Hello, world!")
print(notes.read())

Hello, world!


In [6]:
notes.lookup['write'](text="Shhh... here's a secret: 42")

In [7]:
notes.lookup['read']()

"Shhh... here's a secret: 42"

In [8]:
notes.read()

"Shhh... here's a secret: 42"

## ChatGPT model and MessageHistory

### ChatGPT: `class ChatGPT`

Simple wrapper that keeps a message history object and optional toolkit (TODO: make into list of toolkits).
The object is callable and simply takes a user prompt string and returns the model response string.

### MessageHistory: `class MessageHistory`

A simple class that keeps track of the message history, including the user and assistnat messages, along with system and tool call results. Special subclasses could be defined for things like:
- Rolling history (e.g. last 10 messages)
- Dynamic history based on context (e.g. vector embedding)
- System message that constantly updates (e.g. time, weather, etc.)
- Few-shot learning, i.e. pre-populating the history with some demonstration messages

In [9]:
from gpt_wrapper.models import ChatGPT
from gpt_wrapper.messages import MessageHistory, msg

gpt = ChatGPT(
    model = 'gpt-3.5-turbo',
    toolkit = notes,
    messages=MessageHistory()
)

In [10]:
response = await gpt("What's on my notepad?")
print(response)

[Tool Call]: read {}
[Tool Result]: Shhh... here's a secret: 42
[ChatGPT]: On your notepad, there is a secret number: 42.
On your notepad, there is a secret number: 42.


In [11]:
response = await gpt("Can you double it?")
print(response)

[ChatGPT]: Sure, I can help you with that. Give me a moment.
[Tool Call]: write {
  "text": "The doubled value is 84."
}
[Tool Result]: None
[ChatGPT]: I have doubled the value for you - it is now 84.
I have doubled the value for you - it is now 84.


In [12]:
notes.read()

'The doubled value is 84.'

In [13]:
gpt.messages.history

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': "What's on my notepad?"},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_qJizBworyqr7lh3HX9AB2bZF',
    'function': {'arguments': '{}', 'name': 'read'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': "Shhh... here's a secret: 42",
  'tool_call_id': 'call_qJizBworyqr7lh3HX9AB2bZF'},
 {'content': 'On your notepad, there is a secret number: 42.',
  'role': 'assistant'},
 {'role': 'user', 'content': 'Can you double it?'},
 {'content': 'Sure, I can help you with that. Give me a moment.',
  'role': 'assistant',
  'tool_calls': [{'id': 'call_nhD4CklppuF1gzs0SKl5A5Bl',
    'function': {'arguments': '{\n  "text": "The doubled value is 84."\n}',
     'name': 'write'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': 'None',
  'tool_call_id': 'call_nhD4CklppuF1gzs0SKl5A5Bl'},
 {'content': 'I have doubled the value for you - it is now 84.',
  'role': 'assistant'}]